In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import random
# import SwinTransformerV2  
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from SwinTransformer import SwinTransformer
from SwinTransformerV2 import SwinTransformerV2
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.data import WeightedRandomSampler
from torchvision import transforms
from engine import train
from sklearn.utils.class_weight import compute_class_weight
seed = 42
device = torch.device("cuda")

def set_seeds(seed_value=seed):
    """Set seed for reproducibility."""
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed_value)
    random.seed(seed_value)
device = torch.cuda.is_available() and "cuda" or "cpu"
torch.cuda.manual_seed(seed) or torch.cuda.manual_seed_all(seed)
    
os.environ['CUDA_VISIBLE_DEVICES'] ='0'
print(device)


cuda


In [2]:

# Configuration for data processing
NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 256
IMG_SIZE = 256
transformations = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomApply([
        # Potentially remove or reduce the extent of ColorJitter if dealing with grayscale images
        transforms.RandomAffine(degrees=5, translate=(0.02, 0.02), scale=(0.95, 1.05)),
        transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2)),  # Reduce the maximum sigma
    ], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Adjust if not using pre-trained models or if images are grayscale
])
current_directory = os.getcwd()
training_directory = os.path.join(current_directory, "Dataset", "train")
validation_directory = os.path.join(current_directory, "Dataset", "test")
testing_directory = os.path.join(current_directory, "Dataset", "val")


def setup_dataloaders(train_path, test_path, val_path, transform, batch_size, workers=NUM_WORKERS):
    # Load datasets
    train_dataset = datasets.ImageFolder(train_path, transform=transform)
    test_dataset = datasets.ImageFolder(test_path, transform=transform)
    val_dataset = datasets.ImageFolder(val_path, transform=transform)  
    
    # Calculate class weights (inverse of the class distribution)
    class_sample_counts = [len(np.where(np.array(train_dataset.targets) == i)[0]) for i in np.unique(train_dataset.targets)]
    class_weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
    
    # Compute sample weights
    sample_weights = class_weights[train_dataset.targets]
    
    # Create sampler
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True)

    return train_loader, test_loader, val_loader, class_weights

train_loader, test_loader, val_loader, class_weights = setup_dataloaders(training_directory, testing_directory, validation_directory, transformations, BATCH_SIZE, NUM_WORKERS)
class_weights = class_weights.to(device)
print(device)


cuda


In [5]:
model_config = {
    'embed_dim': 96,
    'depths': [ 2, 2, 6, 2 ],
    'num_heads': [ 2, 4, 8, 16 ],
    'window_size': 8,
    'ape': False, 
    'patch_norm': True,  
    'num_classes': len(class_weights),
    'drop_path_rate': 0.5,
}

# Model, optimizer, and loss function setup
model = SwinTransformerV2(
    img_size=IMG_SIZE, 
    patch_size=32, 
    in_chans=3, 
    **model_config
)
model.to('cuda')
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-06, betas=(0.9, 0.999), weight_decay=1e-2)
loss_function =nn.CrossEntropyLoss(weight=class_weights)
print(device)



cuda


In [6]:
# Print model configuration
print("Model Configuration:")
print(f"  Embedding Dimension: {model_config['embed_dim']}")
print(f"  Depths: {model_config['depths']}")
print(f"  Number of Heads: {model_config['num_heads']}")
print(f"  Window Size: {model_config['window_size']}")
print(f"  Absolute Position Encoding: {model_config['ape']}")
print(f"  Patch Normalization: {model_config['patch_norm']}")
print(f"  Number of Classes: {model_config['num_classes']}")
print(f"  Drop Path Rate: {model_config['drop_path_rate']}")

# Print optimizer and loss function details
print("\nOptimizer and Loss Function:")
print(f"  Learning Rate: {optimizer.defaults['lr']}")
print(f"  Betas: {optimizer.defaults['betas']}")
print(f"  Weight Decay: {optimizer.defaults['weight_decay']}")
print(f"  Loss Function: {loss_function}")

# Print other training parameters
print("\nTraining Parameters:")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Image Size: {IMG_SIZE}")
print(f"  Device: {device}")
print(f"  Number of Epochs: {30}")
print(f"  Learning Rate Scheduler Step Size: {1}")
print(f"  Learning Rate Scheduler Gamma: {1}")


Model Configuration:
  Embedding Dimension: 96
  Depths: [2, 2, 6, 2]
  Number of Heads: [2, 4, 8, 16]
  Window Size: 8
  Absolute Position Encoding: False
  Patch Normalization: True
  Number of Classes: 2
  Drop Path Rate: 0.5

Optimizer and Loss Function:
  Learning Rate: 1e-06
  Betas: (0.9, 0.999)
  Weight Decay: 0.01
  Loss Function: CrossEntropyLoss()

Training Parameters:
  Batch Size: 256
  Image Size: 256
  Device: cuda
  Number of Epochs: 30
  Learning Rate Scheduler Step Size: 1
  Learning Rate Scheduler Gamma: 1


In [7]:
# Training the model
training_results = train(model, train_loader, val_loader, optimizer, loss_function, epochs=30, device=device)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.658464 | train_acc: 0.471540 | val_loss: 1.213098 | val_acc: 0.319754 | lr: 0.00000100
Epoch: 2 | train_loss: 0.511008 | train_acc: 0.566158 | val_loss: 0.842121 | val_acc: 0.471168 | lr: 0.00000100
Epoch: 3 | train_loss: 0.434949 | train_acc: 0.647321 | val_loss: 0.748352 | val_acc: 0.597098 | lr: 0.00000100
Epoch: 4 | train_loss: 0.373820 | train_acc: 0.706411 | val_loss: 0.640981 | val_acc: 0.670573 | lr: 0.00000100
Epoch: 5 | train_loss: 0.332641 | train_acc: 0.762897 | val_loss: 0.558566 | val_acc: 0.736607 | lr: 0.00000100
Epoch: 6 | train_loss: 0.310131 | train_acc: 0.787760 | val_loss: 0.526036 | val_acc: 0.736979 | lr: 0.00000100
Epoch: 7 | train_loss: 0.279436 | train_acc: 0.815724 | val_loss: 0.485411 | val_acc: 0.760975 | lr: 0.00000100
Epoch: 8 | train_loss: 0.263184 | train_acc: 0.834635 | val_loss: 0.486995 | val_acc: 0.759673 | lr: 0.00000100
Epoch: 9 | train_loss: 0.252353 | train_acc: 0.841580 | val_loss: 0.452744 | val_acc: 0.787760 | lr: 0.0